# Calculating FID Score for Merge LoRA model for Flat-Tire Damage only.

If you want to generate images or calculate FID score for other damages or multiple damages, change the "adapter_weights" parameter in function "geberate_images" according to your requirement.

In [1]:
import os
from diffusers import DiffusionPipeline
from tqdm import tqdm


def generate_images(num_images, prompt, output_dir, start):
    
    #Loading the Diffusion Model
    pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to('cuda')

    #Loading Flat Tire LoRA
    pipe.load_lora_weights('IDS-560-Group-3/stable-diffusion-flat-tire-lora-v2', weight_name='pytorch_lora_weights.safetensors', adapter_name='flat-tire')

    #Loading Scratches LoRA
    pipe.load_lora_weights('IDS-560-Group-3/stable-diffusion-scratches-lora-v3', weight_name='pytorch_lora_weights.safetensors', adapter_name='scratches')

    #Loading Dents LoRA
    pipe.load_lora_weights('IDS-560-Group-3/stable-diffusion-dents-lora-v3', weight_name='pytorch_lora_weights.safetensors', adapter_name='dents')

    #Loading Glass Shatter LoRA
    pipe.load_lora_weights('IDS-560-Group-3/stable-diffusion-glass-shatter-lora-v3', weight_name='pytorch_lora_weights.safetensors', adapter_name='glass-shatter')

    #Specifying Adapter Weights
    pipe.set_adapters(['flat-tire','scratches','dents','glass-shatter'], adapter_weights=[0.9,0.1,0.1,0.1])

    #Output Directory
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i in tqdm(range(num_images)):
        image = pipe(prompt).images[0]
        image_path = os.path.join(output_dir, f'flat-tire-generated-{start+i}.png')
        image.save(image_path)

    

c:\Users\Raj\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Generating Multiple images for same prompt

In [2]:
generate_images(36,prompt='a car with front flat tire.',output_dir='./Output/Flat-Tire-FID',start=0)

Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  8.73it/s]
c:\Users\Raj\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Raj\.cache\huggingface\hub\models--IDS-560-Group-3--stable-diffusion-scratches-lora-v3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.war

In [3]:
generate_images(36,prompt='a car with rear flat tire.',output_dir='./Output/Flat-Tire-FID',start=37)

100%|██████████| 36/36 [25:01<00:00, 41.71s/it]


In [4]:
generate_images(36,prompt='a car with broken wheel and front flat tire.',output_dir='./Output/Flat-Tire-FID',start=73)

100%|██████████| 36/36 [25:12<00:00, 42.02s/it]


In [5]:
generate_images(42,prompt='a car with broken wheel and rear flat tire.',output_dir='./Output/Flat-Tire-FID',start=109)

100%|██████████| 42/42 [29:21<00:00, 41.95s/it]


# Real Images

In [1]:
from PIL import Image
import os
import numpy as np

dataset_path = "./Data/Flat Tire Images/"
image_paths = sorted([os.path.join(dataset_path, x) for x in os.listdir(dataset_path)])

real_images = [np.array(Image.open(path).convert("RGB")) for path in image_paths]

In [2]:
from torchvision.transforms import functional as F
import torch


def preprocess_image(image):
    image = torch.tensor(image).unsqueeze(0)
    image = image.permute(0, 3, 1, 2) / 255.0
    return F.center_crop(image, (512, 512))

real_images = torch.cat([preprocess_image(image) for image in real_images])
print(real_images.shape)

torch.Size([150, 3, 512, 512])


# Fake Images

In [3]:
dataset_path = "./Output/Flat-Tire-FID/"
image_paths = sorted([os.path.join(dataset_path, x) for x in os.listdir(dataset_path)])

fake_images = [np.array(Image.open(path).convert("RGB")) for path in image_paths]

fake_images = torch.cat([preprocess_image(image) for image in fake_images])
print(fake_images.shape)

torch.Size([150, 3, 512, 512])


In [4]:
from torchmetrics.image.fid import FrechetInceptionDistance

fid = FrechetInceptionDistance(normalize=True)
fid.update(real_images, real=True)
fid.update(fake_images, real=False)

print(f"FID: {float(fid.compute())}")

c:\Users\Raj\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to C:\Users\Raj/.cache\torch\hub\checkpoints\weights-inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:07<00:00, 12.1MB/s]


FID: 59.84452819824219
